# 自然语言处理实战 —— 文本相似度

在自然语言处理（NLP）过程中，经常会涉及到如何度量两个文本之间的相似性。文本相似度（Semantic Text Similarity）计算就是判断两段文本之间是否相似。相似度有多种粒度的表示方式，可以使用（1,2,3,4,5）中的一个数字表示相似度，值越大表示越相似；也可以使用（0,1）中的一个数字表示相似度，1表示相似，0表示不相似。在本案例中，我们使用（0,1）表示相似度。文本相似度技术可以应用到信息检索、自动问答、机器翻译和自动文摘等NLP任务中。

度量文本相似度包括如下三种方法：

1. 基于关键词匹配的传统方法，如N-gram相似度；


2. 将文本映射到向量空间，再利用余弦相似度等方法；


3. 深度学习的方法，如基于用户点击数据的深度学习语义匹配模型DSSM，基于卷积神经网络的ConvNet等方法。 

本案例中将使用深度学习的 **BERT** 模型进行文本相似度计算。

中文相似度按照长度可以有字与字的相似度、单词与单词的相似度、句子与句子的相似度、段落与段落的相似度和文章与文章的相似度。

本案例主要介绍一种基于词嵌入的中文短句文本相似度计算方法。

### 进入ModelArts

点击如下链接：https://www.huaweicloud.com/product/modelarts.html ， 进入ModelArts主页。点击“立即使用”按钮，输入用户名和密码登录，进入ModelArts使用页面。

### 创建ModelArts notebook

下面，我们在ModelArts中创建一个notebook开发环境，ModelArts notebook提供网页版的Python开发环境，可以方便的编写、运行代码，并查看运行结果。

第一步：在ModelArts服务主界面依次点击“开发环境”、“创建”

![create_nb_create_button](./img/create_nb_create_button.png)

第二步：填写notebook所需的参数：

| 参数 | 说明 |
| - - - - - | - - - - - |
| 计费方式 | 按需计费  |
| 名称 | Notebook实例名称，如 text_sentiment_analysis |
| 工作环境 | Python3 |
| 资源池 | 选择"公共资源池"即可 |
| 类型 | 本案例使用较为复杂的深度神经网络模型，需要较高算力，选择"GPU" |
| 规格 | 选择"8核 &#124; 64GiB &#124; 1*p100" |
| 存储配置 | 选择EVS，磁盘规格5GB |

第三步：配置好notebook参数后，点击下一步，进入notebook信息预览。确认无误后，点击“立即创建”

![create_nb_creation_summary](./img/create_nb_creation_summary.png)

第四步：创建完成后，返回开发环境主界面，等待Notebook创建完毕后，打开Notebook，进行下一步操作。
![modelarts_notebook_index](./img/modelarts_notebook_index.png)

### 在ModelArts中创建开发环境

接下来，我们创建一个实际的开发环境，用于后续的实验步骤。

第一步：点击下图所示的“打开”按钮，进入刚刚创建的Notebook
![inter_dev_env](img/enter_dev_env.png)

第二步：创建一个Python3环境的的Notebook。点击右上角的"New"，然后创建TensorFlow 1.13.1开发环境。

第三步：点击左上方的文件名"Untitled"，并输入一个与本实验相关的名称
![notebook_untitled_filename](./img/notebook_untitled_filename.png)
![notebook_name_the_ipynb](./img/notebook_name_the_ipynb.png)


### 在Notebook中编写并执行代码

在Notebook中，我们输入一个简单的打印语句，然后点击上方的运行按钮，可以查看语句执行的结果：
![run_helloworld](./img/run_helloworld.png)


## 文本相似度计算



### 数据集

本案例采用西安科技大学提供的中文文本相似度语料库。相似度值：（0,1），0表示不相似，1表示相似。

数据格式：

| 字段 | Quality | #1 ID      | #2 ID        | #1 String  | #2 String  |
| ---- | ------- | ---------- | ------------ | ---------- | ---------- |
| 含义 | 相似度  | 第一句编号 | 第二句的编号 | 第一句文本 | 第二句文本 |


### BERT 模型

本实践使用 NLP 领域最新最强大的 **BERT** 模型。

中文**BERT-Base,Chinese**预训练模型，可以从链接[BERT-Base, Chinese](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip)下载使用。

#### 准备源代码和数据

准备案例所需的源代码和数据，相关资源已经保存在 OBS 中，我们通过 ModelArts SDK 将资源下载到本地。

In [1]:
from modelarts.session import Session
session = Session()

if session.region_name == 'cn-north-1':
    bucket_path = 'modelarts-labs/notebook/DL_nlp_text_similarity/text_similarity.tar.gz'
    
elif session.region_name == 'cn-north-4':
    bucket_path = 'modelarts-labs-bj4/notebook/DL_nlp_text_similarity/text_similarity.tar.gz'
else:
    print("请更换地区到北京一或北京四")
    
session.download_data(bucket_path=bucket_path, path='./text_similarity.tar.gz')

!ls -la

Successfully download file modelarts-labs/notebook/DL_nlp_text_similarity/text_similarity.tar.gz from OBS to local ./text_similarity.tar.gz
total 374108
drwxrwxrwx  3 ma-user ma-group      4096 Sep 25 09:37 .
drwsrwsr-x 20 ma-user ma-group      4096 Sep 25 09:27 ..
drwxr-x---  2 ma-user ma-group      4096 Sep 25 09:27 .ipynb_checkpoints
-rw-r-----  1 ma-user ma-group     29752 Sep 25 09:37 text_similarity.ipynb
-rw-r-----  1 ma-user ma-group 383037805 Sep 25 09:37 text_similarity.tar.gz


解压从obs下载的压缩包，解压后删除压缩包。

In [2]:
!tar xf ./text_similarity.tar.gz

!rm ./text_similarity.tar.gz

!ls -la

total 48
drwxrwxrwx  4 ma-user ma-group  4096 Sep 25 09:37 .
drwsrwsr-x 20 ma-user ma-group  4096 Sep 25 09:27 ..
drwxr-x---  2 ma-user ma-group  4096 Sep 25 09:27 .ipynb_checkpoints
drwxr-x---  6 ma-user ma-group  4096 Sep 24 18:12 text_similarity
-rw-r-----  1 ma-user ma-group 29752 Sep 25 09:37 text_similarity.ipynb


#### 导入依赖包

In [3]:
import tensorflow as tf
import os
import csv
import collections
from text_similarity.bert import modeling, optimization, tokenization

tf.logging.set_verbosity(tf.logging.INFO)

#### 定义数据和模型路径

In [4]:
# BERT模型配置文件
bert_config_file = 'text_similarity/chinese_L-12_H-768_A-12/bert_config.json'
vocab_file = 'text_similarity/chinese_L-12_H-768_A-12/vocab.txt'
init_checkpoint = 'text_similarity/chinese_L-12_H-768_A-12/bert_model.ckpt'

# 数据集路径
data_dir = 'text_similarity/data/'

# 模型训练输出位置
output_dir = 'text_similarity/output/'

#### 设置TensorFlow运行相关参数

In [5]:
label_list = ["0", "1"]
do_lower_case = False
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
use_tpu = False
tpu_cluster_resolver = None
master = None


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



#### 设置模型参数

In [6]:
train_batch_size=32
eval_batch_size=8 
predict_batch_size=8
num_epochs = 5.0 
warmup_proportion = 0.1 
learning_rate = 2e-5 
max_seq_length = 128 
save_checkpoints_steps = 1000 
iterations_per_loop = 1000 
num_gpu_cores = 1 

#### 读取BERT预训练模型中文字典

In [7]:
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer.tokenize("今天的天气真好！")

['今', '天', '的', '天', '气', '真', '好', '！']

#### 创建数据输入类

In [8]:
class InputExample(object):

  def __init__(self, guid, text_a, text_b=None, label=None):
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label

class InputFeatures(object):

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example
    
    
class PaddingInputExample(object):
    pass

#### 读取训练数据集

数据集每行的格式为：相似度（Quality），第一句编号（1 ID），第二句的编号（2 ID），第一句文本（1 String），第二句文本（2 String）


In [9]:
def read_tsv(input_file, quotechar=None):
    with tf.gfile.Open(input_file, "r") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            lines.append(line)
            print('line:' , line)
        return lines
    
def create_examples(lines, set_type):
    examples = []
    for (i, line) in enumerate(lines):
      if i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      text_a = tokenization.convert_to_unicode(line[3])
      text_b = tokenization.convert_to_unicode(line[4])
      if set_type == "test":
        label = "0"
      else:
        label = tokenization.convert_to_unicode(line[0])
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

def get_train_examples(data_dir):
    return create_examples(read_tsv(os.path.join(data_dir, "train.tsv")), "train")

train_examples = get_train_examples(data_dir)

line: ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String']
line: ['0', '636', '3175303', '老太太的情绪不稳定。', '这个模式对很多领域都很实用，尤其是数位版权管理方面，因为它的最小描述单元层次，可利用来指定识别码进行个别的筛选与授权使用。']
line: ['0', '922', '4606174', '事情几乎没办成。', '作为拜仁慕尼黑的青训产物，穆勒在2009至10赛季被时任主教练路易斯·范加尔提拔至一线队，他在该赛季几乎参加了队内的所有比赛，为球队赢得了联赛及杯赛双冠王，并且晋身欧洲冠军联赛决赛。']
line: ['0', '387', '1930624', '小明在家务上给妈妈帮了不少忙。', '中华民国的绥远省政府至此彻底消亡。']
line: ['1', '930', '3125', '好不难受。', '很不难受。']
line: ['0', '327', '1633734', '他下午也许来不了。', '从20世界90年代开始，哮喘的得病率在发达国家趋于平稳，而在发展中国家快速增长。']
line: ['1', '832', '2617', '假设真的没有文明和文化，那么这个世界就像个未成品。', '如果真的没有文明和文化，这个世界便像个未成品。']
line: ['1', '804', '2464', '当官不为民做主，不如回家卖红薯。', '假如当官不为民做主，还不如回家卖红薯。']
line: ['0', '801', '4000941', '必须尽快改变现状，否则我真的没有出路了。', '另外，由于每位选手必须要最少投球一次（捕手除外），故参加者多是全能球员。']
line: ['0', '574', '2866598', '路上有许多人在赶路。', '后来，站台上部玻璃板开始松动，考虑到玻璃板可能断裂车站立即紧急疏散。']
line: ['1', '788', '2089', '这一次，投资者仅仅是“有望收回成本”，换句话说，很可能赔本！', '这一次，投资者仅仅是“有望收回成本”，就是说，很可能赔本！']
line: ['0', '569', '2841282', '挑剔的母亲又逼着裁缝把做好的衣服修改了两次。', '银菊露在一般凉茶舖都

line: ['1', '825', '2550', '不管你说什么，他总是听不进去。', '无论你说什么，他还是听不进去。']
line: ['0', '906', '4529736', '好不漂亮。', '外部连结。']
line: ['0', '907', '4534736', '好不热闹。', '滇缅秋海棠']
line: ['0', '622', '3109039', '西兰花菜农已经收割完毕。', '香港路德会派来专机，当飞机沿著跑道起飞时，共军已经朝机场扫射。']
line: ['0', '858', '4288188', '小孙收入微薄，而且上有父母，下有子女，家庭负担不轻，但是，为人慷慨大方，经常帮助比他更困难的朋友。', '但是实习医生的真实情况却是被当作杂工来使唤，平均每天工作16小时以上，月薪仅有三万八千日圆！']
line: ['0', '792', '3956744', '实际上，当地政府多年来对市场别说干预，连辅助性的行为都很少。', '在LMP市场中，存在输电限制的地方，下游需调度价格更高的电源。']
line: ['0', '83', '411462', '信上说的清楚。', '字符串是几乎在所有编程语言中可以实现的非常重要和有用的数据类型。']
line: ['1', '27', '647', '他修好了电脑。', '电脑被他修好了。']
line: ['0', '177', '880363', '她送了我很多礼物。', '姐弟之间。']
line: ['0', '401', '2003383', '这道题就算有点难度，也难不到哪儿去。', '立德洋行于清同治十三年（1875年）创办并经营了一家水厂，其位置位于今天杨树浦水厂的南区，是为杨树浦水厂的前身。']
line: ['0', '878', '4385363', '用用就知道。', '不过休姆男爵夫人始终没有醒来，并于9月3日逝世，终年80岁，她死后遗体安葬于科尔德斯特里姆附近的一个墓园。']
line: ['0', '599', '2991030', '我们这儿冬天没有武汉冷。', '在2004年保罗演讲表示反对废止选举人团制度，他还说若是没有了选举人团制度，将会减弱那些向往自由的州份的投票权力。']
line: ['0', '641', '3201153', '唐代有个诗

line: ['0', '98', '488562', '那人看着都乏味。', '这些古菌特征不是生殖结构，但是它们可以帮助古菌在新的环境下生存。']
line: ['1', '872', '2995', '他逃脱不了法律的制裁，尽管他爸爸是李刚。', '虽说他爸爸是李刚，然而他逃脱不了法律的制裁。']
line: ['1', '362', '1071', '他们家平常不外出旅游。', '平常他们家不外出旅游。']
line: ['1', '770', '2214', '他一边上学，一边打工挣学费。', '他边上学边打工挣学费。']
line: ['0', '912', '4556363', '卖衣服赚钱好容易呀，不到半年我就净落两万多。', '分布在台湾本岛，生长于海拔2，800公尺至3，300公尺的地区，本种种子散布范围约略是以其树高半径的圆圏内，种子发育为小苗后，会形成母树周围有许多小苗的现象，目前已由人工引种栽培。']
line: ['1', '154', '735', '他已经通知了所有人。', '没有人不被他通知。']
line: ['0', '536', '2676565', '这辆车被水淹过。', '此后，在安徽省军事讨伐中国共产党，大花军费、统治苛酷，导致安徽省内各界人士赴南京请愿。']
line: ['0', '512', '2556090', '师傅把车修好了。', '美国协会和国家联盟随即跟进。']
line: ['1', '269', '961', '济南省城，那是大地方，不用说。', '济南省城那是大地方。']
line: ['1', '639', '1892', '那些荔枝他吃了不少。', '那些荔枝他吃了很多。']
line: ['0', '820', '4096116', '他们永远不会得到受害国的原谅，如果他们不真心悔过并道歉的话。', '他们是现存少数的原住民之一。']
line: ['0', '289', '1441830', '整个小区的业主没有他不认识的。', '在古老的新西兰的南北两岛上，因为没有走兽和蛇，鸟类不必逃避，地面上的食物丰富，飞翔能力逐渐退化。']
line: ['0', '228', '1138383', '我写好了信。', '这样，网格计算就提供了一个多用户环境。']
line: ['1', '779',

line: ['0', '909', '4540042', '好不难受。', '翅叶木（学名：）是紫葳科翅叶木属的唯一种。']
line: ['1', '309', '3358', '整个小区的业主没有他不认识的。', '整个小区的业主他都认识。']
line: ['1', '954', '3083', '运动员小张摔了三次，还是没把对方摔倒。', '运动员小张三次都没把对方摔倒。']
line: ['0', '339', '1693787', '我已经很久没有亲自开汽车啦。', '宋代舟师已经作水文测量。']
line: ['0', '610', '3049518', '这种巧克力酒心的。', '波斯湾战争后，美军的回国士兵就不少人发生波斯湾战争症候群的疾病，有人主张贫化铀是造成这种疾病的原因之一，但没有确凿证据。']
line: ['0', '631', '3154926', '老杨这样的大厨红烧鱼算得了什么。', '玩法以回合战略为主，并搭配著全萤幕战斗画面为特色。']
line: ['0', '605', '3024789', '新同学都办完了入学手续。', '卡片游戏。']
line: ['0', '278', '1386313', '你喝了不到两盅酒，就叨叨叨，叨叨叨，你有个够没有？', '江陵焚书被视为中国的文化浩劫之一。']
line: ['1', '521', '1620', '我们班一半人住在三楼。', '三楼，我们班的一半人住在那儿。']
line: ['0', '279', '1391137', '他什么都没个够！', '到1970年代预科教育开始普及，不少学校都开始开设预科课程：']
line: ['0', '857', '4280986', '有人由于不讲逻辑，因此对别人不合逻辑的言论，不但不能觉察它的荒谬，反而随声附和，人云亦云。', '实木地板因为单价高而且又不能耐潮，所以都不建议使用。']
line: ['0', '259', '1290366', '处处我都检查过了。', '而后来迁至美国的移民者并不富裕，和上流社会也没什么联系，所以在宗教、文化等各个方面与最开始的WASP相去甚远。']
line: ['0', '891', '4451065', '教练细心观察、认真分析比赛现场发生的每一个细节。', '马克思主义中，消费主义之所

line: ['0', '230', '1148770', '小王被领导批评的时候，他还在笑呢。', '批评：报刊自由委员会中没有一名新闻学教授，在美国的很多学者眼中，报刊自由主要不是新闻理论问题，而是政治的、经济的、法律的、社会的以及哲学、伦理上的问题。']
line: ['0', '540', '2697909', '在挪威屠杀惨案中，85人被害。', '后来起火点普丁巷附近立了一个纪念碑，高61。']
line: ['0', '898', '4488602', '你不以我们的祖国有着这样的英雄而自豪吗？', '有代表则支持由普鲁士统治的小德意志，不包括任何奥地利领土。']
line: ['0', '566', '2828451', '她去市场买菜回来做饭吃。', '一些著名的艺术人如洪晃、李宗盛等亦先后进驻。']
line: ['0', '164', '819524', '不是每家都买了汽车。', '不过不是所有奇怪的生活经验都属于都市传说，如对第一次见到的事物有奇怪的熟悉感（即既视感）。']
line: ['1', '201', '1040', '他卖了我一只小狗。', '他卖了一只小狗给我。']
line: ['0', '946', '4728572', '小心别擦破手。', '木匠们，他们亦担任时钟镇参事；']
line: ['0', '687', '3431689', '中文系一年级有多少男生呢？', '学校面向全国31个省（直辖市、自治区）和港澳台招生，并招收外国留学生。']
line: ['1', '449', '1538', '他看了一场电影。', '他就看了一场电影。']
line: ['0', '908', '4539789', '好不伤心。', '少脉假卫矛']
line: ['1', '901', '2818', '想说又不敢说。', '既不敢说又想说。']
line: ['0', '231', '1150591', '花猫逮住了一只耗子。', '由于长打能力备受肯定，成为这几年来，中华成棒代表队不动的第四棒，其地位在许多球迷甚至球员心中，有如神一般的存在。']
line: ['0', '139', '693241', '她没有半点私心。', '面对这起严重的车祸，金永炎不仅没有下车，竟然只是从车窗里扔出一张名片留给前来处理事故的警察，然

...

line: ['0', '415', '2073444', '方法始终在变。', '大妄语成，堕无间狱。']
line: ['1', '538', '1657', '把纸撕碎。', '纸被撕碎。']
line: ['0', '45', '221443', '我吃饭吃了八百块钱。', '高桥是清通过日本央行进行赤字开支，并将随之而来的通胀压力最小化。']
line: ['0', '213', '1063744', '那棵树圆叶子。', '这个节目以过度欢闹、肢体搞笑、不时荒唐滑稽的喜剧感而闻名，另外它更会利用布偶角色来制造出独特的戏仿。']
line: ['0', '643', '3213550', '这儿的网速比家里快。', '这类的组织就是如我们所知的Warez软体团队或骇客团队。']
line: ['0', '295', '1473934', '忙的没个空儿。', '于2005年8月16日对外开放。']
line: ['0', '411', '2052274', '这两个问题相同。', '金台区是中国陕西省宝鸡市的一个区，位于市区东北部，名称由境内元末道教建筑金台观而来。']
line: ['1', '47', '398', '我写信写了一个晚上。', '我给他写了一个通宵的信。']
line: ['0', '99', '494112', '那东西闻着都恶心。', '一个婴儿出生时生理的的性别和长大后对自己的性别认同不一定相同。']
line: ['1', '704', '1870', '演讲比赛谁报名了？', '谁报名参加演讲比赛了？']
line: ['0', '491', '2450792', '飞碟！', '他的诗打破了早期白话诗单纯直白的单调性。']
line: ['0', '523', '2614681', '我能把这块大石头举起来。', '日高郡，为日本的郡：']
line: ['0', '881', '4401458', '有两只小鸡争着饮水，蹬翻了水碗，往青石板上一跳，满石板印着许多小小的“个”字。', '现代遗传生物学者认为，许多先天性格存在著基因的影响。']
line: ['0', '124', '616313', '他这个人没个正经的。', '相思的恋人会请月神来睹誓或鉴察评理。']
line: ['0', '60', '299776',

line: ['0', '660', '3297274', '小吃摊儿旁边围着很多人。', '终身学习，继续教育，在职进修，缓解过度拥挤的校园，已经是上世纪七十年代初的重要课题。']
line: ['0', '280', '1399316', '没个错，你不用骗自己。', '去看过弥留的母亲之后，他知道自己还爱著卡门。']
line: ['0', '233', '1164025', '明天他们上广州。', '重庆武斗中甚至使用了军舰，涪陵地区武斗因为当地驻军介入炮舰轰击城区，完全是正规战争的打法。']
line: ['1', '117', '240', '缺不了你的钱花。', '少不了你花的钱。']
line: ['1', '121', '729', '我才不干呢。', '我不干，哼。']
line: ['0', '823', '4112013', '由于对造成事故的主观原因认识不足，因此导致了此类事故的再次发生。', '在斋藤龙兴继位后，西美浓三人众陆续背叛斋藤氏加入织田氏，导致斋藤氏被织田氏攻灭。']
line: ['0', '369', '1844736', '柱子让汽车给撞倒啦！', '天津一汽丰田汽车有限公司简称天津一汽丰田，是由中国第一汽车集团公司、天津一汽夏利汽车股份有限公司、丰田汽车股份有限公司和丰田汽车（中国）投资有限公司共同出资组建的汽车制造公司。']
line: ['0', '102', '507482', '房子粉刷一新。', '联合国大会']
line: ['0', '450', '2245303', '他一路上没说话。', '由纽西兰的搁浅记录显示，牠们在南太平洋西部可能亦有分布。']
line: ['1', '847', '2919', '因为时间太仓促了，他们放弃了这次参赛的机会。', '因时间太仓促了，他们放弃了这次参赛的机会。']
line: ['0', '757', '3783026', '不应该只看到个别动作像不像，而是要看舞蹈中表现的情感可能不可能存在。', '一些网站会使用计数器来禁止广告过滤软件的使用，如尝试检测是否存在广告过滤器并提示用户关闭软件，或直接阻止用户访问网页直至他们关闭广告过滤软件。']
line: ['0', '72', '358444', '眼睛看坏了。', '这些不同风格的元素以各种别具一格的方式结合

line: ['1', '61', '838', '刚让岳父岳母给关进洞房就让你这丫头片子给提溜出来了。', '刚让岳父岳母给关进洞房就被你这丫头片子给提溜出来了。']
line: ['0', '306', '1529069', '她讨了个没趣儿。', '他退出政界回到他在纽约州威斯特彻斯特郡的庄园上。']
line: ['1', '778', '2425', '不应该只看到个别动作像不像，而是要看舞蹈中表现的情感可能不可能存在。', '是要看舞蹈中表现的情感可能不可能存在，而不应该只看到个别动作像不像。']
line: ['1', '109', '59', '吃一点是一点。', '吃一口是一口。']
line: ['0', '745', '3720916', '雷锋精神不但没有过时，而且永远不会过时。', '虽然政治上议会党团的定义没有最低人数限制，但各国议会的运作规程中通常只承认达到一定人数的议会党团：例如澳大利亚国会规定的最低人数是（两院合计）五人，而加拿大国会规定的最低人数是众议院12人或参议院五人。']
line: ['0', '578', '2885242', '你可以回到学校找师弟师妹们帮忙。', '另外，学校之中国语文科以普通话授课。']
line: ['0', '191', '950025', '他们家跑了一条狗。', '从最后一个冰河时期开始，人类已经就到洛矶山脉地区，主要是美洲印第安人的个部落，包括肖肖尼、苏、犹他、阿帕契、黑脚、平头等，他们在洛矶山麓和河谷猎捕猛犸象和原始美洲野牛（比后来的美洲野牛要大20%），他们可能和后来的印第安人一样，在秋季和冬季移居到平原地带，在春季和夏季进入山区捕鱼、猎鹿和采集浆果、块根等。。']
line: ['0', '969', '4842909', '你知道个屁！', '参考资料。']
line: ['1', '920', '2438', '风遇到防护林，速度就减少了百分之七十到八十。到了距离防护林等于林木高度二十米的地方，风速又恢复原来的强度。所以防护林必须是并行排列的许多林带，两列之间的距离不要超过林木高度的二十倍。', '风的速度遇到防护林就减少了百分之七十到八十。到了距离为林木高度二十倍的地方，风速又恢复原来的强度。以致于防护林必须是并行排列的许多林带，两列之间的距离不要超过林木高度的二十倍。']
l

line: ['0', '180', '896137', '他抢了我一张邮票。', '武松也因此受到牵连，被赶出衙门。']
line: ['0', '972', '4856598', '你难道不是老师？', '早前詹森再批准美军攻击海防及河内更多目标，但禁止攻击区继续保留。']
line: ['0', '899', '4493954', '风遇到防护林，速度就减少了百分之七十到八十。到了距离防护林等于林木高度二十米的地方，风速又恢复原来的强度。所以防护林必须是并行排列的许多林带，两列之间的距离不要超过林木高度的二十倍。', '圣马力诺是世界杯资格赛最快入球的保持者，于1993年对英格兰赛事中，开赛后8。']
line: ['0', '286', '1428250', '也没有个长远留下人的理。', '但孙中山有顾虑，故没有派代表出席此次会议，然而答应对张西曼的建议将详加考虑。']
line: ['0', '219', '1091931', '我写好信的时候她已经睡着了。', '在一晚上的睡眠剥夺之后患者可能感到情绪明显变得轻松。']
line: ['1', '427', '1201', '他懒得去逛公园。', '他不想去逛公园。']
line: ['0', '499', '2493169', '这种树叶子很大。', '在这种场合，无线接入点成为使用者端接入有线网路的一个接口。']
line: ['0', '672', '3356598', '吃了这剂药，过两天就会好的。', '从甲府风林收购在天皇杯有好良表现的门将佐藤优也，在1月球队在季前9场热身赛当中保持不败。']
line: ['0', '953', '4762124', '他怀疑事情还没有办完。', '人类与及其他动物都没有方法排出多余的铁质。']
line: ['0', '870', '4345282', '不成也值。', '颜师古之弟。']
line: ['1', '560', '1581', '在挪威屠杀惨案中，85人被害。', '挪威屠杀惨案中85人遇害。']
line: ['0', '815', '4074025', '就算前面是地雷阵，我也要一往直前。', '在这层保护之下，细菌大量繁殖。']
line: ['0', '220', '1097959', '来客人的家庭。', '每集有100位参赛者

line: ['1', '861', '2869', '村民自发组织起巡逻队，以防不测。', '村民自发组织起巡逻队，免得不测。']
line: ['0', '138', '686282', '他没往家里寄过半分钱。', '首长人数依村社规模而有所不同，如大社3位、中社2位、小社1位，首长若是年纪过大，荷兰人也会多选1位首长以辅助。']
line: ['0', '401', '2002251', '这道题就算有点难度，也难不到哪儿去。', '幼儿的记忆广度更为有限，通常是4±1个单元。']
line: ['0', '641', '3204015', '唐代有个诗人。这个诗人叫李白。', '恶搞版本。']
line: ['0', '127', '632959', '他的数学不怎么好。', '另外一些优惠是因为行政之便而限制优惠只给予使用八达通的乘客。']
line: ['0', '891', '4451092', '教练细心观察、认真分析比赛现场发生的每一个细节。', '在消费社会中，更被强调的是个人主义，也就是个人作为一个具有自主性的能动者，依工具理性达成自己的目标，例如消费行为代表个人表达的欲望。']
line: ['0', '254', '1267688', '我羊肉不吃，吃牛肉。', '气候风力。']
line: ['0', '273', '1361238', '您要是再给她一个不痛快，我就把您这两年做的事都告诉哥哥。', '海峡中间偏近伊朗的一边有一个大岛叫做格什姆岛，隶属于伊朗。']
line: ['0', '157', '781962', '她的笑便渐渐少了。', '至于穿梭机，则因为最初的节目中，主持人是在身处太空中的穿梭机的驾驶舱内执行任务，因此称为430穿梭机。']
line: ['0', '556', '2777096', '学校里调来了一位女教师。', '热夫拉尔被视为海地最好的总统之一，他支持教育，建立了医药、航海、艺术诸学院，鼓励在城市中建立工业技术学校，还派遣学生去欧洲留学。']
line: ['0', '418', '2085971', '没有喝完的半杯水。', '拘留可能伴随着逮捕，但也可能没有。']
line: ['0', '568', '2835282', '那声音引导着她暂时与厂区生活拉开了距离。', '人类一直就将鲎当作食物，

#### 转换为 BERT 输入向量

打印前5个样例文本，及其字向量、文本向量、位置向量和标签。

In [10]:
def truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_id=0,
            is_real_example=False)
    
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]") # 句头添加 [CLS] 标志
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]") # 句尾添加[SEP] 标志
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)  
    input_mask = [1] * len(input_ids)

    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    label_id = label_map[example.label]
    
    if ex_index < 5:
        tf.logging.info("*** Example ***")
        tf.logging.info("guid: %s" % (example.guid)) 
        tf.logging.info("tokens: %s" % " ".join([tokenization.printable_text(x) for x in tokens])) 
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))  
        tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask])) 
        tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids])) 
        tf.logging.info("label: %s (id = %d)" % (example.label, label_id)) 

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id=label_id,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_file):
    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer)
        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["label_ids"] = create_int_feature([feature.label_id])
        features["is_real_example"] = create_int_feature([int(feature.is_real_example)])

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()

train_file = os.path.join(output_dir, "train.tf_record")

file_based_convert_examples_to_features(train_examples, label_list, max_seq_length, tokenizer, train_file)

INFO:tensorflow:Writing example 0 of 9416
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-1
INFO:tensorflow:tokens: [CLS] 老 太 太 的 情 绪 不 稳 定 。 [SEP] 这 个 模 式 对 很 多 领 域 都 很 实 用 ， 尤 其 是 数 位 版 权 管 理 方 面 ， 因 为 它 的 最 小 描 述 单 元 层 次 ， 可 利 用 来 指 定 识 别 码 进 行 个 别 的 筛 选 与 授 权 使 用 。 [SEP]
INFO:tensorflow:input_ids: 101 5439 1922 1922 4638 2658 5328 679 4937 2137 511 102 6821 702 3563 2466 2190 2523 1914 7566 1818 6963 2523 2141 4500 8024 2215 1071 3221 3144 855 4276 3326 5052 4415 3175 7481 8024 1728 711 2124 4638 3297 2207 2989 6835 1296 1039 2231 3613 8024 1377 1164 4500 3341 2900 2137 6399 1166 4772 6822 6121 702 1166 4638 5033 6848 680 2956 3326 886 4500 511 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

#### 加载模型参数，构造模型结构

In [11]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    output_layer = model.get_pooled_output()
    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        probabilities = tf.nn.softmax(logits, axis=-1)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)

def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):

  def model_fn(features, labels, mode, params):

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    is_real_example = None
    if "is_real_example" in features:
      is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
    else:
      is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
        num_labels, use_one_hot_embeddings)

    tvars = tf.trainable_variables()
    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None

    if mode == tf.estimator.ModeKeys.TRAIN:

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)

    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits, is_real_example):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(
            labels=label_ids, predictions=predictions, weights=is_real_example)
        loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits, is_real_example])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)

    else:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          predictions={"probabilities": probabilities},
          scaffold_fn=scaffold_fn)
    return output_spec
  return model_fn


num_train_steps = int(len(train_examples) / train_batch_size * num_epochs)
num_warmup_steps = int(num_train_steps * warmup_proportion)

model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=init_checkpoint,
    learning_rate=learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=use_tpu,
    use_one_hot_embeddings=use_tpu)

#### 模型训练

In [12]:
def file_based_input_fn_builder(input_file, seq_length, is_training, drop_remainder):

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    def _decode_record(record, name_to_features):
        example = tf.parse_single_example(record, name_to_features)

        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t
        return example

    def input_fn(params):
        batch_size = params["batch_size"]

        d = tf.data.TFRecordDataset(input_file)        
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d
    return input_fn


run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=master,
    model_dir=output_dir,
    save_checkpoints_steps=save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=iterations_per_loop,
        num_shards=num_gpu_cores,
        per_host_input_for_training=is_per_host))

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=max_seq_length,
    is_training=True,
    drop_remainder=False) 


estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=train_batch_size,
    eval_batch_size=eval_batch_size,
    predict_batch_size=predict_batch_size)

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:Using config: {'_model_dir': 'text_similarity/output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f06475178d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=N

INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  nam

INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_11/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = output_weights:0, shape = (2, 768)
INFO:tensorflow:  name = output_bias:0, shape = (2,)
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into text_similarity/output/model.ckpt.
INFO:tensorflow:global_step/sec: 1.88037
INFO:tensorflow:examples/sec: 60.1717
INFO:tensorflow:global_step/sec: 2.08161
INFO:tensorflow:exam

#### 读取验证集

In [13]:
def get_dev_examples(data_dir):
    return create_examples(read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

eval_examples = get_dev_examples(data_dir)
print(type(eval_examples))
print(eval_examples[0])

eval_file = os.path.join(output_dir, "eval.tf_record")

file_based_convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer, eval_file)

line: ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String']
line: ['0', '662', '3307482', '他花光了钱。', '崇祯十四年（1641年），李自成数次围攻开封，丁启睿督催左良玉、虎大威、杨德政、方国安、傅宗龙等人率兵解围。']
line: ['0', '662', '3306833', '他花光了钱。', '这种抗生素的临床实验开始于1960年代，并成功的治疗急性白血病和淋巴瘤。']
line: ['1', '818', '2516', '有我杨某在，你就别想翻天！', '只要有我杨某在，你就别想翻天！']
line: ['0', '559', '2791731', '余德利抬头发现李东宝的目光很慌。', '后母回家后，发现叶限抱树而睡，便没有追究。']
line: ['0', '828', '4138697', '我不常逛街，因为我老没时间。', '齐格飞（Siegfried，齐格鲁德的德语写法，为同一人）在杀掉法夫纳时就全身浴血，但因为有一片树叶黏在背后，所以造成有一小块皮肤没有沾到血，而成为他唯一的弱点。']
line: ['0', '78', '387909', '嘴干死了。', '此文先后对诗，赋，碑，诔，铭，箴，颂，论，奏，说十种进行分析。']
line: ['1', '881', '2386', '塑料不腐烂分解是一大长处，因为当塑料垃圾被深埋时，他永远不会变成任何有毒的化学物质污染人类生存的环境，而且即便被焚烧，大部分塑料也不会释放出有毒气体。', '塑料垃圾被深埋时，他永远不会变成任何有毒的化学物质污染人类生存的环境，而且即便被焚烧，大部分塑料也不会释放出有毒气体，故塑料不腐烂分解是一大长处。']
line: ['0', '421', '2101313', '静静地坐着。', '这个发现令许多人想进一步了解海马区在记忆及学习机制的作用，因而成为一种流行，无论在神经解剖学、生理学、行为学等等各种不同领域，都对海马区做了相当丰富的研究。']
line: ['0', '846', '4228944', '阳春四月，平原地区的桃花早就凋谢了，可是这里却仍然是一片绯红，桃花含苞欲放，艳丽多姿。', '在大陆地区共有2500多名员工，研发中心位于苏州和广州。']
lin

line: ['0', '571', '2851399', '地震与海啸令这个国家的经济又倒退了十年。', '工作涉及的国家。']
line: ['0', '134', '665926', '有冤没处说。', '满洲国康德元年（1934年）十二月，地方行政机构改革中划满洲为十四省，方正县划归三江省。']
line: ['1', '125', '432', '他玩起来没个完。', '他玩起来没个度。']
line: ['0', '888', '4437283', '每个人都有属于自己的一条路，或者平坦，或者艰险，或者漫长，或者短暂；每个人的爱情经历都是值得回味的，或者甜蜜，或者苦涩，或者如愿，或者遗憾······。', '语义互操作性，又称为语义协同工作能力或者语义互用性，是互操作性的一种层次。']
line: ['0', '252', '1255533', '他北京话说的很好？', '如渤海被封为忽汗州大都督、疏勒被封为疏勒都督等等。']
line: ['0', '934', '4669647', '你再摔摔，看这次的动作合不合标准。', '分布于巴基斯坦、印度以及中国大陆的新疆等地。']
line: ['1', '40', '490', '这幅画花了我一个多月。', '我用了一个月的时间画这幅画。']
line: ['0', '712', '3556431', '请进！', '石墨烯的结构非常稳定，碳碳键仅为1。']
line: ['0', '460', '2299266', '谁也不知道事情的真相。', '因此各界鼓励孩童使用维基百科找资料，也可能会让大多数学童看到维基百科的色情图片。']
line: ['0', '661', '3303514', '墙上挂着齐白石的画。', '贵州龙与趾龙的下颞孔长而狭窄，延伸至头骨的后段，其他肿肋龙科没有这种特征。']
line: ['1', '642', '1905', '西兰花菜农已经收割完毕。', '菜农已经把西兰花收割完毕。']
line: ['0', '998', '4988066', '我没打过老婆。', '在制作电影的同时，冯仍是新鸿基证券的执行董事。']
line: ['0', '202', '1008954', '东西全给他吃光了。', '玩法如下：']
line: ['0', '935', '467248

INFO:tensorflow:Writing example 0 of 2000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: dev-1
INFO:tensorflow:tokens: [CLS] 他 花 光 了 钱 。 [SEP] 崇 祯 十 四 年 （ 164 ##1 年 ） ， 李 自 成 数 次 围 攻 开 封 ， 丁 启 睿 督 催 左 良 玉 、 虎 大 威 、 杨 德 政 、 方 国 安 、 傅 宗 龙 等 人 率 兵 解 围 。 [SEP]
INFO:tensorflow:input_ids: 101 800 5709 1045 749 7178 511 102 2300 4875 1282 1724 2399 8020 10048 8148 2399 8021 8024 3330 5632 2768 3144 3613 1741 3122 2458 2196 8024 672 1423 4729 4719 998 2340 5679 4373 510 5988 1920 2014 510 3342 2548 3124 510 3175 1744 2128 510 987 2134 7987 5023 782 4372 1070 6237 1741 511 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


line: ['0', '522', '2607468', '你把衣服洗了吧。', '小酒店（"L\'Assommoir"，或译酒店、小酒馆等），法国写实主义作家左拉的卢贡-马卡尔家族系列小说的第七部，发表于1877年，是一部研究酗酒后果的小说，把第二帝国时期巴黎下层人的生活描写得灵魂活现，一开始是在公共福利报上连载，引起轩然大波。']
line: ['0', '547', '2732971', '中间有一间小屋子。', '马路最初以碎石铺路，至1920年代起改为沥青路面。']
line: ['0', '912', '4558444', '卖衣服赚钱好容易呀，不到半年我就净落两万多。', '中华民国。']
line: ['1', '301', '1359', '准是瞎指挥，没个不出事儿。', '肯定是瞎指挥，一定会出事儿。']
line: ['0', '109', '540034', '办一件放心一件。', '九月廿八至十月八日合共九日内，官涌之战六役，林则徐役役皆胜，将英军驱逐尖沙咀。']
line: ['1', '872', '2996', '他逃脱不了法律的制裁，尽管他爸爸是李刚。', '虽说他爸爸是李刚，他却逃脱不了法律的制裁。']
line: ['0', '716', '3579013', '咱们开始吧！', '由于我对那份文件的内容有特别的印象，自从摩门教的骚动开始后，我也常常同朋友们谈到这件事，并且清楚记得那些纸上别的什么都有，可就没有埃及的象形文字。']
line: ['0', '395', '1970624', '他不见得就弄懂了。', '国际台的重新定位也是为了减低采购节目的成本，和吸引高收入及高教育水平之观众。']
line: ['0', '494', '2466282', '再棘手的事情他都有办法解决。', '花岗石匾额和门框，入口木门刻有门神，而前厅挡中顶部置有一块饰有花卉雕刻图案和中国民间故事的彩门。']
line: ['0', '213', '1062909', '那棵树圆叶子。', '乌龙派出所动画版']
line: ['1', '407', '1517', '小明在家务上给妈妈帮了不少忙。', '在家务上小明给妈妈帮了很多忙。']
line: ['0', '174', '865042', '他吃螃蟹吃病了。', '平原是海拔较低的平

#### 在验证集上验证模型，评估结果

In [14]:
num_actual_eval_examples = len(eval_examples)

tf.logging.info("***** Running evaluation *****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(eval_examples), num_actual_eval_examples,
                len(eval_examples) - num_actual_eval_examples)
tf.logging.info("  Batch size = %d", eval_batch_size)


eval_input_fn = file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=False)

result = estimator.evaluate(input_fn=eval_input_fn, steps=None)

print("\n打印文本相似度评估指标")
for key in result:
    print(key+' : '+str(result[key]))


INFO:tensorflow:***** Running evaluation *****
INFO:tensorflow:  Num examples = 2000 (2000 actual, 0 padding)
INFO:tensorflow:  Batch size = 8
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNor

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FRO

INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert


打印文本相似度评估指标
eval_accuracy : 0.988
eval_loss : 0.04225879
loss : 0.04225879
global_step : 1471


### 在线测试

由以上训练得到模型进行在线测试，可以任意输入两个句子，进行相似度分析。

任意一个句子未输入，则结束在线文本相似度分析。

In [15]:
from text_similarity.bert import similarity
sim = similarity.BertSim()

print("在线测试\n")
sim.set_mode(tf.estimator.ModeKeys.PREDICT)
predict = 1
while predict is not None:
    sentence1 = input('\n输入句子1: ')
    sentence2 = input('\n输入句子2: ')
    predict = sim.predict(sentence1, sentence2)
    if predict is not None:
        print('\n相似度是：{}'.format(predict[0][1]))

INFO:tensorflow:Using config: {'_model_dir': './text_similarity/output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.9
  allow_growth: true
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f061bcc6f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


在线测试



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./text_similarity/output/model.ckpt-1471
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.



输入句子1: 我曾经帮这位教授整理过稿子。

输入句子2: 这位教授的稿子我帮着整理过。


INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 我 曾 经 帮 这 位 教 授 整 理 过 稿 子 。 [SEP] 这 位 教 授 的 稿 子 我 帮 着 整 理 过 。 [SEP]
INFO:tensorflow:input_ids: 101 2769 3295 5307 2376 6821 855 3136 2956 3146 4415 6814 4943 2094 511 102 6821 855 3136 2956 4638 4943 2094 2769 2376 4708 3146 4415 6814 511 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)



相似度是：0.9998441934585571

输入句子1: 我习惯喝咖啡不放糖。

输入句子2: 他边打工挣学费边上学。


INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 我 习 惯 喝 咖 啡 不 放 糖 。 [SEP] 他 边 打 工 挣 学 费 边 上 学 。 [SEP]
INFO:tensorflow:input_ids: 101 2769 739 2679 1600 1476 1565 679 3123 5131 511 102 800 6804 2802 2339 2914 2110 6589 6804 677 2110 511 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)



相似度是：7.943964737933129e-05

输入句子1: 

输入句子2: 

再见
